In [1]:
import pandas as pd
import numpy as np
import os as os

In [2]:
cwd= os.getcwd()
data_list = os.listdir(os.getcwd()+'\data')
data_list

['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

In [3]:
df_list=[]
for i in data_list:
    df_list.append((i.split('.')[0]))
df_list


['olist_customers_dataset',
 'olist_geolocation_dataset',
 'olist_orders_dataset',
 'olist_order_items_dataset',
 'olist_order_payments_dataset',
 'olist_order_reviews_dataset',
 'olist_products_dataset',
 'olist_sellers_dataset',
 'product_category_name_translation']

In [4]:
for i in range(len(data_list)):
    df_list[i]=pd.read_csv(os.getcwd()+'\\data\\'+data_list[i])
    

In [68]:
df_list[2].shape
df_list[7].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## 1. Total order count with revenue for each states?

In [82]:
a1=pd.merge(df_list[0],df_list[2], on='customer_id',how='right').merge(df_list[4],on='order_id',how='inner')

a1.groupby('customer_state').agg({'order_id':'count', 'payment_value': 'sum'})\
.reset_index()\
.rename(columns={'order_id':'order_count','payment_value':'order_value'})\
.sort_values('order_value',ascending=False)\
.head(5)


,customer_state,order_count,order_value
25,SP,43622,5998226.96
18,RJ,13527,2144379.69
10,MG,12102,1872257.26
22,RS,5668,890898.54
17,PR,5262,811156.38


## 2. Total order count with revenue for each states where review_score of order is 5?

In [83]:
a2=pd.merge(df_list[0],df_list[2], on='customer_id',how='right')\
     .merge(df_list[4],on='order_id',how='inner')\
     .merge(df_list[5],on='order_id',how='inner')
a2= a2[a2['review_score']==5]
a2.groupby('customer_state').agg({'order_id':'count', 'payment_value': 'sum'})\
    .reset_index()\
    .rename(columns={'order_id':'order_count','payment_value':'order_value'})\
    .sort_values('order_value',ascending=False)\
    .head(5)





,customer_state,order_count,order_value
25,SP,26252,3526218.66
18,RJ,7299,1122959.96
10,MG,7114,1077242.14
22,RS,3307,509489.71
17,PR,3143,462279.12


## 3. Total order count made by each seller where payment mode was "debit_card"?

In [85]:
a3=pd.merge(df_list[2],df_list[3], on='order_id',how='inner')\
     .merge(df_list[4],on='order_id',how='inner')\
     .merge(df_list[7],on='seller_id',how='inner')
a3= a3[a3['payment_type']=='debit_card']
a3.groupby('seller_id').agg({'order_id':'count', 'payment_value': 'sum'})\
    .reset_index()\
    .rename(columns={'order_id':'no_of_debit_card_mode_sales','payment_value':'order_value'})\
    .sort_values('no_of_debit_card_mode_sales',ascending=False)\
    .head(5)





,seller_id,no_of_debit_card_mode_sales,order_value
288,6560211a19b47992c3666cc44a7e94c0,31,2619.95
427,955fee9216a65b617aa5c0531780ce60,30,3489.35
399,8b321bb669392f5163d04c59e235e066,29,885.97
616,e9779976487b77c6d4ac45f75ec7afe9,26,1544.12
103,1f50f920176fa81dab994f9023523100,25,2684.86


## 4. Top 5 english name of products with maximum selling price?

In [113]:
df_list[8].head() #product_category_name
df_list[6].head() #product_category_name
df_list[3].head() #product_id

a4=pd.merge(df_list[3],df_list[6], on='product_id',how='inner')\
     .merge(df_list[8],on='product_category_name',how='inner')
a4.groupby('product_category_name_english')['price']\
.max()\
.sort_values(ascending=False)\
.head(5)

product_category_name_english
housewares             6735.00
computers              6729.00
art                    6499.00
small_appliances       4799.00
musical_instruments    4399.87
Name: price, dtype: float64

## 5. Year (order_items.shipping_limit_date) with maximum order ordered?

In [140]:
a5=df_list[3]
a5.groupby(pd.to_datetime(a5['shipping_limit_date']).dt.year)['order_id'].agg('count')

shipping_limit_date
2016      370
2017    49765
2018    62511
2020        4
Name: order_id, dtype: int64